In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransTokenizer
%cd IndicTransTokenizer
!python3 -m pip install --editable ./
%cd ..

Restart runtime Before Running below Code/Cell

In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [2]:
def initialize_model_and_tokenizer(ckpt_dir, direction, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = IndicTransTokenizer(direction=direction)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            src=True,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text
        generated_tokens = tokenizer.batch_decode(generated_tokens.detach().cpu().tolist(), src=False)

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

# **Loading model for English to indian Language translation**

In [3]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, "en-indic", quantization)

ip = IndicProcessor(inference=True)



config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/4.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

Test example

In [4]:
en_sents = [
    "When I was young, I used to go to the park every day.",
    "He has many old books, which he inherited from his ancestors.",
]
#Source language and target language
src_lang, tgt_lang = "eng_Latn", "guj_Gujr"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

# printing line by line
print(f"\n{src_lang} - {tgt_lang}")
for input_sentence, translation in zip(en_sents, hi_translations):
    print(f"{src_lang}: {input_sentence}")
    print(f"{tgt_lang}: {translation}")

# flush the models to free the GPU memory
# del en_indic_tokenizer, en_indic_model


eng_Latn - guj_Gujr
eng_Latn: When I was young, I used to go to the park every day.
guj_Gujr: જ્યારે હું નાનો હતો, ત્યારે હું દરરોજ બગીચામાં જતો હતો.
eng_Latn: He has many old books, which he inherited from his ancestors.
guj_Gujr: તેમની પાસે ઘણા જૂના પુસ્તકો છે, જે તેમને તેમના પૂર્વજો પાસેથી વારસામાં મળ્યા હતા.


Load Data Files for all the Required Language ( English , Gujarati , Hindi)

In [5]:
# load english dataset
file_path = "/kaggle/input/dataset-subset/en_1000_ground_truth.txt"
#load english dataset
with open(file_path, "r", encoding="utf-8") as file:
    en_text = file.readlines()

# print(en_text)

In [6]:
print(en_text[0])

On the side-lines of this event, I hope the delegates joining us from abroad, shall have some time to see the history and splendour of Delhi.



In [7]:
# load gujarati dataset
file_path = "/kaggle/input/dataset-subset/gu_1000_ground_truth.txt"
#load gujarati dataset
with open(file_path, "r", encoding="utf-8") as file:
    gu_text = file.readlines()

# print(gu_text)

In [21]:
print(gu_text[1])

In [8]:
# load hindi dataset
file_path = "/kaggle/input/dataset-subset/hi_1000_ground_truth.txt"
#load english dataset
with open(file_path, "r", encoding="utf-8") as file:
    hi_text = file.readlines()

# print(hi_text)

In [10]:
print(hi_text[0])

मुझे उम्मीद है कि विदेशों से आए प्रतिनिधियों के पास दिल्ली के इतिहास और गौरव को देखने के लिए कुछ समय मिलेगा।



English To Gujarati

In [11]:
#Source language and target language
src_lang, tgt_lang = "eng_Latn", "guj_Gujr"
gu_translations = batch_translate(en_text, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)


In [12]:
gu_translations[0]

'આ કાર્યક્રમની સાથે સાથે હું આશા રાખું છું કે વિદેશથી અમારી સાથે જોડાનારા પ્રતિનિધિઓને દિલ્હીનો ઇતિહાસ અને ભવ્યતા જોવા માટે થોડો સમય મળશે.'

In [13]:

Eng_guj_tran = 'eng_guj_1000_indictran.txt'
# Open the output file in write mode
with open(Eng_guj_tran, "w", encoding="utf-8") as file:
    for translation in gu_translations:
        # Write each translation to the file
        file.write(translation + "\n")

print("Translations file :",Eng_guj_tran)


Translations file : eng_guj_1000_indictran.txt


# **Loading model for Indian to English Language translation**

Gujarati to English

In [27]:
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, "indic-en", "")

ip = IndicProcessor(inference=True)


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [ ]:
#Source language and target language
src_lang, tgt_lang = "guj_Gujr","eng_Latn"
gutoen_translations = batch_translate(gu_text, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)


In [31]:
gutoen_translations[0]

'Along with this occasion, I hope that the delegates joining this ceremony from abroad will also spend some time to see the history and grandeur of Delhi.'

In [32]:

    guj_eng_tran = 'guj_eng_1000_indictran.txt'
    # Open the output file in write mode
    with open(guj_eng_tran, "w", encoding="utf-8") as file:
        for translation in gutoen_translations:
            # Write each translation to the file
            file.write(translation + "\n")

    print("Translations file :",guj_eng_tran)


Translations file : guj_eng_1000_indictran.txt


# **Loading model for Indian to Indian Language translation**

indian lang to other indian lang

In [14]:
indic_indic_ckpt_dir = "ai4bharat/indictrans2-indic-indic-1B"  # ai4bharat/indictrans2-indic-indic-dist-320M
indic_indic_tokenizer, indic_indic_model = initialize_model_and_tokenizer(indic_indic_ckpt_dir, "indic-indic", quantization)

ip = IndicProcessor(inference=True)


config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

configuration_indictrans.py:   0%|          | 0.00/14.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_indictrans.py:   0%|          | 0.00/61.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-indic-1B:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Gujarati to Hindi

In [15]:
#Source language and target language
src_lang, tgt_lang =  "guj_Gujr","hin_Deva"
# gu_translations = batch_translate(en_text, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
gutohi_translations = batch_translate(gu_text, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)


In [16]:
gutohi_translations[0]

'इस अवसर के साथ, मुझे उम्मीद है कि विदेशों से समारोह में शामिल होने वाले प्रतिनिधि दिल्ली के इतिहास और भव्यता को देखने के लिए भी कुछ समय बिताएंगे।'

In [25]:
guj_hin_tran = 'guj_hin_1000_indictran.txt'
# Open the output file in write mode
with open(guj_hin_tran, "w", encoding="utf-8") as file:
    for translation in gutohi_translations:
        # Write each translation to the file
        file.write(translation + "\n")

print("Translations file :",guj_hin_tran)

Translations file : guj_hin_1000_indictran.txt


Hindi to Gujarati

In [18]:
#Source language and target language
src_lang, tgt_lang =  "hin_Deva","guj_Gujr"
# gu_translations = batch_translate(en_text, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
hitogu_translations = batch_translate(hi_text, src_lang, tgt_lang, indic_indic_model, indic_indic_tokenizer, ip)


In [22]:
hitogu_translations[0]

'હું આશા રાખું છું કે વિદેશથી આવેલા પ્રતિનિધિઓ પાસે દિલ્હીના ઇતિહાસ અને ગૌરવને જોવા માટે થોડો સમય હશે.'

In [26]:
hin_guj_tran = 'hin_guj_1000_indictran.txt'
# Open the output file in write mode
with open(hin_guj_tran, "w", encoding="utf-8") as file:
    for translation in hitogu_translations:
        # Write each translation to the file
        file.write(translation + "\n")

print("Translations file :",hin_guj_tran)


Translations file : hin_guj_1000_indictran.txt
